In [1]:
import re
import urllib
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
TICKER = 'CSCO' # 주식명 설정
URL_TEMPLATE = "https://feeds.finance.yahoo.com/" + \
    "rss/2.0/headline?s=%s%region=US&lang=en-US"

In [4]:
nltk.download('stopwords') # 불용어를 내려받는다. 불용어란 제외하는 편이 나은 단어

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Taejun/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
def get_article_urls(ticker):
    # 해당 주식과 관련된 기사를 반환한다.
    link_pattern = re.compile(r"<link>[^<]*</link>")
    xml_url = URL_TEMPLATE % ticker
    xml_data = urllib.request.urlopen(xml_url).read().decode('utf-8')
    link_hits = re.findall(link_pattern, xml_data)
    return [h[6:-7] for h in link_hits]

In [21]:
def get_article_content(url):
    '''입력: 신문기사 url
    출력: 신문 기사 전처리 결과
    HTML 파일을 내려받은 뒤
    각 문단의 내용을 정리한다.
    ''' 
    paragraph_re = re.compile(r"<p>.*</p>")
    tag_re = re.compile(r"<[^>]*>")
    raw_html = urllib.request.urlopen(url).read().decode('utf-8')
    paragraphs = re.findall(paragraph_re, raw_html)
    all_text = "".join(paragraphs)
    content = re.sub(tag_re, "", all_text)
    return content

In [22]:
def text_to_bag(txt):
    '''입력: 문자열
    출력: BoW(Bag-of-words) 특징값
    불용어(stop words)는 무시하고 처리한다.'''
    lemmatizer = WordNetLemmatizer()
    txt_as_ascii = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt_as_ascii)
    words = [t for t in tokens if t.isalpha()]
    lemmas = [lemmatizer.lemmatize(w) for w in words]
    # lemmatization은 다양한 형태로 등장하는 같은 단어를 처리하는 과정
    stop = set(stopwords.words('english'))
    nostops = [l for l in lemmas if l not in stop]
    return nltk.FreqDist(nostops)

In [23]:
def count_good_bad(bag):
    '''입력: BoW 특징값
    출력: 긍정/부정적 단어의 개수'''
    # BoW는 자연어 처리에서 널리 쓰이는 가장 기본적인 특징값으로, 문장이나 문서 등 문자열 데이터를 벡터로 표현
    good_synsets = set(wn.synsets('good')+wn.synsets('up'))
    bad_synsets = set(wn.synsets('bad')+wn.synsets('donw'))
    n_good, n_bad = 0,0
    for lemma, ct in bag.items():
        ss = wn.synsets(lemma)
        if good_synsets.intersection(ss): n_good += 1
        if bad_synsets.intersection(ss): n_bad += 1
    return n_good, n_bad

In [25]:
urls = get_article_urls(TICKER)
contents = [get_article_content(u) for u in urls]
bags = [text_to_bag(txt) for txt in contents]
counts = [count_good_bad(txt) for txt in bags]
n_good_articles = len([_ for g, b in counts if g > b])
n_bad_articles = len([_ for g, b in counts if g < b])

print("긍정적인 기사: %i개, 부정적인 기사: %i개" %
     (n_good_articles, n_bad_articles))

TypeError: not enough arguments for format string

In [ ]:
# 품사 태깅
nltk.pos_tag(["I", "drink", "milk"])
nltk.help.upenn_tagset()

In [ ]:
# cosine similarity
# 두 벡터를 비교할때 많이 씀. 벡터 내적 연산의 정의에 의해 크기가 1인 벡터의 내적은 두 벡터가 이루는 각도의 코사인 값이 됩니다. 
# 그 결과 동일한 벡터의 코사인 유사도는 1이 나오고 단어가 전혀 겹치지 않으면 0이 나옵니다.
# BoW 특징값 벡터는 모든 성분이 양수이기 때문에 0보다 작은 코사인 유사도는 나오지 않습니다.